In [1]:
import torch
import json
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import HfApi
from huggingface_hub import upload_file, delete_file,upload_folder,repository


C:\Users\DESKTOP\anaconda3\envs\a2\lib\site-packages\huggingface_hub\snapshot_download.py:11: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  FutureWarning,


In [2]:
from huggingface_hub import notebook_login
#huggingface upload
notebook_login()

Login successful
Your token has been saved to C:\Users\DESKTOP/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
def readKLUETrainData(path):
    trainDataset=[]
    with open(path,'rb')as file:
        STSdata=json.load(file)
    
    for item in STSdata:
        score=float(item['labels']['label'])/5.0

        inputEx=InputExample(
        texts=[item["sentence1"],item["sentence2"]],
        label=score
        )
        trainDataset.append(inputEx)
    
    return trainDataset

In [4]:
def readNLITrainData(path):
    trainInput=[]
    trainDataset=pd.DataFrame({"sentence1":[],
                              "sentence2":[],
                              "score":[]})
    df = pd.read_csv(dataPath, on_bad_lines='skip')
    
    trainDataset["sentence1"]=df["sentence1"]
    trainDataset["sentence2"]=df["sentence2"]
    trainDataset["score"]=df["score"]
    try:
        for item in trainDataset.loc():
            inputEx=InputExample(
            texts=[item["sentence1"],item["sentence2"]],
            label=float(item["score"]/5.0)
            )
            trainInput.append(inputEx)
    except:
        pass

    return trainInput

In [5]:
#검증용
def readModuTrainData(path):
    trainDataset=[]
    with open(dataPath,'rb')as file:
        STSdata=json.load(file)
        
    for item in STSdata['data']:
        paraphrases=item["paraphrases"]
        sens=list()
        for form in paraphrases:
            sentence=form["form"]
            sens.append(sentence)

        for idx,value in enumerate(sens):
            for k in range(idx+1,len(sens)):
                inputEx=InputExample(
                    texts=(sens[idx],sens[k]),
                    label=float(0.9)
                )
                trainDataset.append(inputEx)
    return trainDataset

In [6]:

#NLI sts 데이터
dataPath="./dataset/sts-train.csv"
trainDataset=readNLITrainData(dataPath)

In [7]:
BATCH_SIZE=32
EPOCH=4
outputPath=("TAACO_STS_epoch-"+str(EPOCH)).replace("/","-")
outputPath='./outputs/'+outputPath

In [8]:
embedding_model = models.Transformer(
    model_name_or_path="KDHyun08/TAACO_STS", 
    max_seq_length=256,
    do_lower_case=True
)

pooling_model = models.Pooling(
    embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False,
)

In [9]:
model = SentenceTransformer(modules=[embedding_model, pooling_model])

In [10]:
trainDataLoader=DataLoader(
trainDataset,
shuffle=True,
batch_size=BATCH_SIZE
)
trainLoss=losses.CosineSimilarityLoss(model=model)

In [11]:
evaluator=EmbeddingSimilarityEvaluator.from_input_examples(
trainDataset,
name="sts-vali"
)

In [12]:
model.fit(
train_objectives=[(trainDataLoader, trainLoss)],
    evaluator=evaluator,
    epochs=EPOCH,
    evaluation_steps=len(trainDataset),
    output_path=outputPath
 )

C:\Users\DESKTOP\anaconda3\envs\a2\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

Iteration:   0%|          | 0/142 [00:00<?, ?it/s]

TypeError: 'float' object is not subscriptable

In [ ]:
#tokenizer.save_pretrained("./model")
model.save("./model")

In [ ]:
upload_folder(
    folder_path ="model",
    path_in_repo="",
    repo_id="KDHyun08/TAACO_STS",
)